<a href="https://colab.research.google.com/github/yangr296/ECE539/blob/main/ECE539_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import tensorflow as tf
import numpy as np
import random
from sklearn.model_selection import train_test_split

with open('/Users/publicenemy989/Downloads/parkinsons/parkinsons.data', 'r') as f:
  tmp = np.genfromtxt(f,delimiter=',')
np.set_printoptions(precision=3,suppress=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data Preprocessing

In [27]:
from tensorflow import keras

X, y = tmp[1:, 1:], tmp[1:, -7]
X = np.delete(X, 16, 1)
X = X.astype(float)
"""
index by columns
X[0] MDVP:Fo(Hz)
X[1] MDVP:Fhi(Hz)
X[2] MDVP:Flo(Hz)
X[3] MDVP:Jitter(%)
X[4] MDVP:Jitter(Abs)
X[5] MDVP:RAP
X[6] MDVP:PPQ
X[7] Jitter:DDP
X[8] MDVP:Shimmer
X[9] MDVP:Shimmer(dB)
X[10] Shimmer:APQ3
X[11] Shimmer:APQ5
X[12] MDVP:APQ
X[13] Shimmer:DDA
X[14] NHR
X[15] HNR
X[16] RPDE
X[17] DFA
X[18] spread1
X[19] spread2
X[20] D2
X[21] PPE
"""

'\nindex by columns\nX[0] MDVP:Fo(Hz)\nX[1] MDVP:Fhi(Hz)\nX[2] MDVP:Flo(Hz)\nX[3] MDVP:Jitter(%)\nX[4] MDVP:Jitter(Abs)\nX[5] MDVP:RAP\nX[6] MDVP:PPQ\nX[7] Jitter:DDP\nX[8] MDVP:Shimmer\nX[9] MDVP:Shimmer(dB)\nX[10] Shimmer:APQ3\nX[11] Shimmer:APQ5\nX[12] MDVP:APQ\nX[13] Shimmer:DDA\nX[14] NHR\nX[15] HNR\nX[16] RPDE\nX[17] DFA\nX[18] spread1\nX[19] spread2\nX[20] D2\nX[21] PPE\n'

In [28]:
from scipy.stats import pearsonr
pair = []
for i in range(21):
  for j in range((i + 1), 22):
    a = X[:, i]
    b = X[:, j]
    corr, _ = pearsonr(a, b)
    if(corr > 0.95):
      pair.append([i, j])
      print("features %i and %i have corr of %f" %(i, j, corr))
print(pair)

features 3 and 5 have corr of 0.990276
features 3 and 6 have corr of 0.974256
features 3 and 7 have corr of 0.990276
features 5 and 6 have corr of 0.957317
features 5 and 7 have corr of 1.000000
features 6 and 7 have corr of 0.957319
features 8 and 9 have corr of 0.987258
features 8 and 10 have corr of 0.987625
features 8 and 11 have corr of 0.982835
features 8 and 12 have corr of 0.950083
features 8 and 13 have corr of 0.987626
features 9 and 10 have corr of 0.963198
features 9 and 11 have corr of 0.973751
features 9 and 12 have corr of 0.960977
features 9 and 13 have corr of 0.963202
features 10 and 11 have corr of 0.960070
features 10 and 13 have corr of 1.000000
features 11 and 13 have corr of 0.960072
features 18 and 21 have corr of 0.962435
[[3, 5], [3, 6], [3, 7], [5, 6], [5, 7], [6, 7], [8, 9], [8, 10], [8, 11], [8, 12], [8, 13], [9, 10], [9, 11], [9, 12], [9, 13], [10, 11], [10, 13], [11, 13], [18, 21]]


Function def

In [29]:
from sklearn.metrics import accuracy_score
def performSVM(X, y):
  aveScore = 0
  for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    aveScore += np.round(accuracy_score(y_test, y_pred), 3)
  aveScore /= 5
  return aveScore

In [30]:
from sklearn import svm
deleted = []
for i in pair:
  if i[0] in deleted:
    print("skipped pair(%i, %i), because %i already deleted" %(i[0], i[1], i[0]))
    continue
  if i[1] in deleted:
    print("skipped pair(%i, %i), because %i already deleted" %(i[0], i[1], i[1]))
    continue
  A = X
  B = X
  A = np.delete(A, i[0], 1)
  B = np.delete(B, i[1], 1)
  score_a = performSVM(A, y)
  score_b = performSVM(B, y)
  if(score_a > score_b):
    delete = i[1]
  else:
    delete = i[0]
  deleted.append(delete)
  print("Accuracy: %i(%.3f) %i(%.3f) Delete %i" %(i[0], score_a, i[1], score_b, delete))

Accuracy: 3(0.856) 5(0.785) Delete 5
Accuracy: 3(0.851) 6(0.836) Delete 6
Accuracy: 3(0.830) 7(0.774) Delete 7
skipped pair(5, 6), because 5 already deleted
skipped pair(5, 7), because 5 already deleted
skipped pair(6, 7), because 6 already deleted
Accuracy: 8(0.841) 9(0.795) Delete 9
Accuracy: 8(0.856) 10(0.805) Delete 10
Accuracy: 8(0.780) 11(0.821) Delete 8
skipped pair(8, 12), because 8 already deleted
skipped pair(8, 13), because 8 already deleted
skipped pair(9, 10), because 9 already deleted
skipped pair(9, 11), because 9 already deleted
skipped pair(9, 12), because 9 already deleted
skipped pair(9, 13), because 9 already deleted
skipped pair(10, 11), because 10 already deleted
skipped pair(10, 13), because 10 already deleted
Accuracy: 11(0.821) 13(0.775) Delete 13
Accuracy: 18(0.816) 21(0.841) Delete 18


In [109]:
Corr = [[0] * 2 for i in range(22)]
for i in range(22):
  feature = X[:, i]
  corr, _ = pearsonr(feature, y)
  Corr[i][0] = abs(corr)
  Corr[i][1] = i
  print("Pearson's Correlation of feature", i, "is: %.3f" % corr)

Pearson's Correlation of feature 0 is: -0.384
Pearson's Correlation of feature 1 is: -0.166
Pearson's Correlation of feature 2 is: -0.380
Pearson's Correlation of feature 3 is: 0.278
Pearson's Correlation of feature 4 is: 0.339
Pearson's Correlation of feature 5 is: 0.267
Pearson's Correlation of feature 6 is: 0.289
Pearson's Correlation of feature 7 is: 0.267
Pearson's Correlation of feature 8 is: 0.367
Pearson's Correlation of feature 9 is: 0.351
Pearson's Correlation of feature 10 is: 0.348
Pearson's Correlation of feature 11 is: 0.351
Pearson's Correlation of feature 12 is: 0.364
Pearson's Correlation of feature 13 is: 0.348
Pearson's Correlation of feature 14 is: 0.189
Pearson's Correlation of feature 15 is: -0.362
Pearson's Correlation of feature 16 is: 0.309
Pearson's Correlation of feature 17 is: 0.232
Pearson's Correlation of feature 18 is: 0.565
Pearson's Correlation of feature 19 is: 0.455
Pearson's Correlation of feature 20 is: 0.340
Pearson's Correlation of feature 21 is: 

In [110]:
# sort all the correlations
num = 22
Corr.sort()
print("ranking the %i lowest correlations: " %num)
for i in range(num):
  print("feature: %2d corr: %.3f" %(Corr[i][1], Corr[i][0]))

ranking the 22 lowest correlations: 
feature:  1 corr: 0.166
feature: 14 corr: 0.189
feature: 17 corr: 0.232
feature:  7 corr: 0.267
feature:  5 corr: 0.267
feature:  3 corr: 0.278
feature:  6 corr: 0.289
feature: 16 corr: 0.309
feature:  4 corr: 0.339
feature: 20 corr: 0.340
feature: 13 corr: 0.348
feature: 10 corr: 0.348
feature:  9 corr: 0.351
feature: 11 corr: 0.351
feature: 15 corr: 0.362
feature: 12 corr: 0.364
feature:  8 corr: 0.367
feature:  2 corr: 0.380
feature:  0 corr: 0.384
feature: 19 corr: 0.455
feature: 21 corr: 0.531
feature: 18 corr: 0.565


In [111]:
# drop the features with the lowest correlation
num = 17
print("X shape before drops:", X.shape)
print("Corr shape:", len(Corr))
for i in range(num):
  X = np.delete(X, Corr[i][1], 1)
  # update feature positions
  for j in Corr:
    if j[1] > i:
      j[1] = j[1] - 1
print("X shape after drops:", X.shape)

X shape before drops: (195, 22)
Corr shape: 22
X shape after drops: (195, 5)


In [112]:
# partition into training and validation set at 80/20 division
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
# Converting y_r and y_t into one-hot encoding using a keras utility
#  to_categorical

# print(y_test)
#[1, 0] is 0
#[0, 1] is 1
y_test = keras.utils.to_categorical(y_test)
y_train = keras.utils.to_categorical(y_train)
# print(y_test)

In [113]:
from sklearn.impute import KNNImputer

if np.sum(np.isnan(X)):
  print('Total of NaN before imputation:', np.sum(np.isnan(X)))
  imputer = KNNImputer(n_neighbors=2, weights="uniform")
  X1 = imputer.fit_transform(X)
  print('Total of NaN after imputation:', np.sum(np.isnan(X1)))
else:
  X1 = X
  print('no NaN')

no NaN


In [114]:
from sklearn import preprocessing
print("before transformation\n--------------------")
for i in range(X.shape[1]):
  print("range of feature %i is %i"%(i, np.ptp(X_train[:,i])))
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-5, 5))
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)
print("after transformation\n--------------------")
for i in range(X.shape[1]):
  print("range of feature %i is %i"%(i, np.ptp(X_train[:,i])))

# standardize X_train and X_test
scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
print(X_train.shape)

before transformation
--------------------
range of feature 0 is 171
range of feature 1 is 0
range of feature 2 is 0
range of feature 3 is 1
range of feature 4 is 5
after transformation
--------------------
range of feature 0 is 10
range of feature 1 is 10
range of feature 2 is 10
range of feature 3 is 10
range of feature 4 is 10
(156, 5)


Model Definition

In [115]:
from keras import layers
net = tf.keras.models.Sequential([
    tf.keras.Input(shape=(5,)),
    tf.keras.layers.GaussianNoise(0.1),
    tf.keras.layers.experimental.RandomFourierFeatures(output_dim=4096, scale=10.0, kernel_initializer="gaussian"),
    layers.Dense(256, activation = 'relu'),
    tf.keras.layers.LayerNormalization(),
    layers.Dense(256, activation = 'relu'),
    tf.keras.layers.LayerNormalization(),
    layers.Dense(256, activation = 'relu'),
    tf.keras.layers.LayerNormalization(),
    layers.Dense(256, activation = 'relu'),
    tf.keras.layers.LayerNormalization(),
    layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(2, activation='softmax'),
    ])
"""
tf.keras.layers.Dense(8, input_dim=5, activation = 'relu'),
tf.keras.layers.LayerNormalization(),
tf.keras.layers.Dense(32, activation = 'relu'),
tf.keras.layers.LayerNormalization(),
tf.keras.layers.Dense(128, activation = 'relu'),
tf.keras.layers.Dense(256, activation = 'relu'),
tf.keras.layers.LayerNormalization(),
tf.keras.layers.Dense(128, activation = 'relu'),
tf.keras.layers.LayerNormalization(),
tf.keras.layers.Dense(32, activation = 'relu'),
tf.keras.layers.LayerNormalization(),
tf.keras.layers.Dense(2, activation='softmax')
"""


"\ntf.keras.layers.Dense(8, input_dim=5, activation = 'relu'),\ntf.keras.layers.LayerNormalization(),\ntf.keras.layers.Dense(32, activation = 'relu'),\ntf.keras.layers.LayerNormalization(),\ntf.keras.layers.Dense(128, activation = 'relu'),\ntf.keras.layers.Dense(256, activation = 'relu'),\ntf.keras.layers.LayerNormalization(),\ntf.keras.layers.Dense(128, activation = 'relu'),\ntf.keras.layers.LayerNormalization(),\ntf.keras.layers.Dense(32, activation = 'relu'),\ntf.keras.layers.LayerNormalization(),\ntf.keras.layers.Dense(2, activation='softmax')\n"

In [116]:
# compile the keras model
net.compile(loss='binary_crossentropy', optimizer='Adadelta',
              metrics=['accuracy'])

Training

In [120]:
# fit the keras model on the dataset
net.fit(X_train, y_train, epochs=100, batch_size=39)

Epoch 1/100
4/4 [==============================] - 0s 29ms/step - loss: 0.2549 - accuracy: 0.9167
Epoch 2/100
4/4 [==============================] - 0s 29ms/step - loss: 0.2522 - accuracy: 0.9103
Epoch 3/100
4/4 [==============================] - 0s 28ms/step - loss: 0.2508 - accuracy: 0.9167
Epoch 4/100
4/4 [==============================] - 0s 27ms/step - loss: 0.2487 - accuracy: 0.9167
Epoch 5/100
4/4 [==============================] - 0s 38ms/step - loss: 0.2495 - accuracy: 0.9231
Epoch 6/100
4/4 [==============================] - 0s 27ms/step - loss: 0.2507 - accuracy: 0.8910
Epoch 7/100
4/4 [==============================] - 0s 33ms/step - loss: 0.2464 - accuracy: 0.9038
Epoch 8/100
4/4 [==============================] - 0s 30ms/step - loss: 0.2458 - accuracy: 0.9167
Epoch 9/100
4/4 [==============================] - 0s 28ms/step - loss: 0.2455 - accuracy: 0.9038
Epoch 10/100
4/4 [==============================] - 0s 28ms/step - loss: 0.2430 - accuracy: 0.9295
Epoch 11/100
4/4 [=

Testing

In [123]:
# Evaluate the trained model using keras built-in function
score = net.evaluate(X_test, y_test, verbose=0)
print("Test loss:", format(score[0],".4f"))
print("Test accuracy:", score[1])

Test loss: 0.3083
Test accuracy: 0.8717948794364929


In [124]:
# Evaluate predicted testing output
# Since softmax output is used, these outputs are probability
# vectors of value between 0 and 1 and values of each output
# vector added to 1
y_softmax = net.predict(X_test)
# y_pc gives the indices (strating from 0) of the max elements
y_pc = np.argmax(y_softmax, axis = -1)
# convert y_pc into one-hot encoding
y_pred = keras.utils.to_categorical(y_pc)

Cmat = tf.math.confusion_matrix(y_test.argmax(axis=-1),y_pred.argmax(axis=-1))
print(Cmat)

2/2 [==============================] - 0s 8ms/step
tf.Tensor(
[[ 9  4]
 [ 1 25]], shape=(2, 2), dtype=int32)
